In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
import re

In [2]:
image_path = '../data/extracted_letters'
files = os.listdir(image_path)
X = list()
for file in files:
    path = os.path.join(image_path, file)
    y = re.findall('/(\w)_\d+\.\w+', path)[0]
    img = Image.open(path)
#     r_image = img.resize((24, 24))
    array = list(np.array(img).flatten())
    array.append(y)
    X.append(array)
df = pd.DataFrame(X)
df.columns = ['pixel{}'.format(i) for i in range(1, 577)] + ['label']

In [3]:
df.head()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel424,pixel425,pixel426,pixel427,pixel428,pixel429,pixel430,pixel431,pixel432,label
0,0,4,5,0,12,0,0,3,0,8,...,4,8,3,0,0,0,10,3,0,N
1,4,0,16,0,0,0,6,0,4,10,...,1,0,9,0,0,0,0,0,0,V
2,0,0,0,0,0,0,0,3,0,8,...,7,0,4,0,9,0,0,0,0,r
3,7,0,0,9,1,6,12,7,2,10,...,5,9,0,0,2,0,5,0,0,r
4,0,2,0,0,5,9,0,10,8,0,...,0,8,0,7,14,3,0,1,0,t


In [4]:
df.to_csv('../data/cnn_data_sets.csv', index=None)

In [11]:
X = df.iloc[:, 0:432].values

In [12]:
y = df.iloc[:, -1].values

In [16]:
# 特征处理
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [24]:
from sklearn.preprocessing import OneHotEncoder
oh = OneHotEncoder()
y = oh.fit_transform(np.expand_dims(y, 1))

In [26]:
y = y.toarray()

In [29]:
# shuffle the datasets
np.random.seed(1)

In [31]:
# shuffle the datasets
index = np.random.permutation(X.shape[0])
X = X[index]
y = y[index]

In [34]:
y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [35]:
# split datasets to train and test sets
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [42]:
# transform data to 0~1 distribution
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()
X_train_mm = mm.fit_transform(X_train)
X_test_mm = mm.transform(X_test)

/home/liuchao/.pyenv/versions/3.6.3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [46]:
# resize the data to image size
X_train_final = X_train_mm.reshape([-1, 24, 18, 1])
X_test_final = X_test_mm.reshape([-1, 24, 18, 1])

In [48]:
# save the data to local files
import pickle
pickle.dump(X_train_final, open('../data/X_train_cnn', 'wb'), protocol=4)
pickle.dump(X_test_final, open('../data/X_test_cnn', 'wb'), protocol=4)
pickle.dump(y_train, open('../data/y_train_cnn', 'wb'), protocol=4)
pickle.dump(y_test, open('../data/y_test_cnn', 'wb'), protocol=4)
pickle.dump(le, open('../data/le_cnn', 'wb'), protocol=4)
pickle.dump(oh, open('../data/oh_cnn', 'wb'), protocol=4)

In [61]:
pickle.dump(mm, open('../data/mm_cnn', 'wb'), protocol=4)

In [49]:
y_train.shape

(193051, 47)

In [50]:
import tensorflow as tf

In [52]:
# 输入与输出
X = tf.placeholder(tf.float32, shape=[None, 24, 18, 1], name='x-input')
y_ = tf.placeholder(tf.int32, shape=[None, 47], name='y-input')
y_ = tf.cast(y_, 'float')

# 卷积层1
w_conv1 = tf.Variable(tf.truncated_normal(shape=[5, 5, 1, 6], stddev=0.1))
bias_conv1 = tf.Variable(tf.constant(0.1, shape=[6]))
h_conv1 = tf.nn.relu(tf.nn.conv2d(X, w_conv1, padding='SAME', strides=[1,1,1,1]) + bias_conv1)
# 池化层1
h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# 卷积层2
w_conv2 = tf.Variable(tf.truncated_normal(shape=[5, 5, 6, 16], stddev=0.1))
bias_conv2 = tf.Variable(tf.constant(0.1, shape=[16]))
h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, w_conv2, padding='VALID', strides=[1,1,1,1]) + bias_conv2)
# 池化层2
# h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# 全连接层1
W_fc1 = tf.Variable(tf.truncated_normal(shape=[8 * 5 * 16, 120], stddev=0.1))
b_fc1 = tf.Variable(tf.constant(0.1, shape=[120]))
h_conv2_flat = tf.reshape(h_conv2, [-1, 8*5*16])
h_fc1 = tf.nn.relu(tf.matmul(h_conv2_flat, W_fc1) + b_fc1)
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
w1_loss = tf.contrib.layers.l2_regularizer(0.001)(W_fc1)

# 全连接层2
W_fc2 = tf.Variable(tf.truncated_normal(shape=[120, 84], stddev=0.1))
b_fc2 = tf.Variable(tf.constant(0.1, shape=[84]))
h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
w2_loss = tf.contrib.layers.l2_regularizer(0.001)(W_fc2)
# keep_prob2 = tf.placeholder(tf.float32)
# h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob2)

# 输出层
W_fc3 = tf.Variable(tf.truncated_normal(shape=[84, 47], stddev=0.1))
b_fc3 = tf.Variable(tf.constant(0.1, shape=[47]))
y_conv = tf.nn.softmax(tf.matmul(h_fc2, W_fc3) + b_fc3)
w3_loss = tf.contrib.layers.l2_regularizer(0.001)(W_fc3)

# global_step = tf.Variable(0, trainable=False)
# learning_rate = tf.train.exponential_decay(
# 0.0001,
# global_step,
# X_train.shape[0] / 100,
# 0.99
# )
# 损失函数与准确率
cross_entropy = -tf.reduce_sum(y_*tf.log(tf.clip_by_value(y_conv, 1e-10, 1.0)))
losses = cross_entropy + w1_loss + w2_loss + w3_loss
train_step = tf.train.AdamOptimizer(1e-3).minimize(losses)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [53]:
sess = tf.InteractiveSession()

In [54]:
sess.run(tf.initialize_all_variables())

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [56]:
batch_size = 1000
data_size = X_train_final.shape[0]
for i in range(30000):
    start = (i*batch_size) % data_size # 可以重复选取
    end = min(start+batch_size, data_size)
    if i % 1000 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={X: X_train_final[start:end], y_: y_train[start:end],
                                                       keep_prob:1.0})
        print('training accuracy after %d step(s) is %.2f' % (i, train_accuracy))
    sess.run(train_step, feed_dict={X: X_train_final[start:end], y_: y_train[start:end], keep_prob:0.5})
test_accuracy = accuracy.eval(feed_dict={X: X_test_final, y_: y_test, keep_prob:1.0})
print("test accuracy: %.2f" % test_accuracy)

training accuracy after 0 step(s) is 0.03
training accuracy after 1000 step(s) is 0.99
training accuracy after 2000 step(s) is 0.98
training accuracy after 3000 step(s) is 0.99
training accuracy after 4000 step(s) is 0.99
training accuracy after 5000 step(s) is 0.99
training accuracy after 6000 step(s) is 0.99
training accuracy after 7000 step(s) is 0.98
training accuracy after 8000 step(s) is 0.99
training accuracy after 9000 step(s) is 0.99
training accuracy after 10000 step(s) is 1.00
training accuracy after 11000 step(s) is 0.99
training accuracy after 12000 step(s) is 0.99
training accuracy after 13000 step(s) is 0.99
training accuracy after 14000 step(s) is 0.99
training accuracy after 15000 step(s) is 0.99
training accuracy after 16000 step(s) is 0.99
training accuracy after 17000 step(s) is 0.99
training accuracy after 18000 step(s) is 0.99
training accuracy after 19000 step(s) is 0.99
training accuracy after 20000 step(s) is 0.99
training accuracy after 21000 step(s) is 0.99
t

In [57]:
saver = tf.train.Saver()

In [62]:
saver.save(sess, "./cnn_model_0.99.ckpt")

'./cnn_model_0.99.ckpt'

In [71]:
y_conv

<tf.Tensor 'Softmax_1:0' shape=(?, 47) dtype=float32>

In [72]:
X

<tf.Tensor 'x-input_1:0' shape=(?, 24, 18, 1) dtype=float32>

In [73]:
keep_prob

<tf.Tensor 'Placeholder_1:0' shape=<unknown> dtype=float32>